In [ ]:
!pip install pycuda  # Instala la librería PyCUDA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=4844349b53e32a8865fe8373b4bdcddd3b3065044731074eab27ddecff03b0d4
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
!pip install Bio  # Instala la librería Biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from Bio import SeqIO  # Importa el módulo SeqIO de Biopython para manipular archivos de secuencias
import numpy as np  # Importa la librería numpy para manipulación de arrays

def merge_sequences_from_fasta(file_path, max_length=None):
    sequences = []  # Inicializa una lista para almacenar las secuencias
    total_length = 0  # Inicializa la longitud total de las secuencias
    for record in SeqIO.parse(file_path, "fasta"):  # Itera sobre cada registro en el archivo FASTA
        seq = str(record.seq)  # Convierte la secuencia a una cadena
        sequences.append(seq)  # Agrega la secuencia a la lista
        total_length += len(seq)  # Suma la longitud de la secuencia a la longitud total
        if max_length and total_length >= max_length:  # Si se alcanza la longitud máxima, se detiene
            break
    merged_sequence = "".join(sequences)  # Une todas las secuencias en una sola cadena
    if max_length:
        return merged_sequence[:max_length]  # Retorna la secuencia unida truncada a la longitud máxima
    return merged_sequence  # Retorna la secuencia unida completa

def sequence_to_matrix(sequence, num_columns):
    # Asegurarse de que la secuencia tenga suficiente longitud para llenar las columnas
    padded_sequence = sequence.ljust(num_columns, 'N')  # Completa con 'N' si es necesario
    matrix = np.array(list(padded_sequence)).reshape(-1, num_columns)  # Convierte la secuencia a una matriz
    return matrix  # Retorna la matriz


In [ ]:
file_path_1 = "/content/drive/MyDrive/SEMESTRE 2024-1/PROGRAMACION CONCURRENTE Y DISTRIBUIDA/Proyecto/Proyecto/E_coli.fna"
file_path_2 = "/content/drive/MyDrive/SEMESTRE 2024-1/PROGRAMACION CONCURRENTE Y DISTRIBUIDA/Proyecto/Proyecto/Salmonella.fna"

max_length = 10000  # Define una longitud máxima de 10,000 nucleótidos

merged_sequence_1 = merge_sequences_from_fasta(file_path_1, max_length)  # Une las secuencias del primer archivo
merged_sequence_2 = merge_sequences_from_fasta(file_path_2, max_length)  # Une las secuencias del segundo archivo


In [ ]:
# Para inicializar automáticamente PyCUDA y la GPU
import pycuda.autoinit

# Para interactuar directamente con el controlador de la GPU
import pycuda.driver as cuda

# Para manipular arrays y operaciones en la CPU
import numpy as np

# Para compilar y cargar módulos CUDA en la GPU
from pycuda.compiler import SourceModule

# Para medir los tiempos
import time

# Para generar gráficos
import matplotlib.pyplot as plt

In [ ]:
def create_dotplot(seq1, seq2):
    len1, len2 = len(seq1), len(seq2)  # Obtiene las longitudes de las secuencias

    dotplot_matrix = np.zeros((len1, len2), dtype=np.int32)  # Inicializa la matriz dotplot con ceros

    kernel_code = """
    __global__ void compare_sequences(char* seq1, char* seq2, int* dotplot, int len1, int len2) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        int idy = blockIdx.y * blockDim.y + threadIdx.y;
        if (idx < len1 && idy < len2) {
            dotplot[idx * len2 + idy] = (seq1[idx] == seq2[idy]) ? 1 : 0;
        }
    }
    """

    mod = SourceModule(kernel_code)  # Compila el código CUDA
    compare_sequences = mod.get_function("compare_sequences")  # Obtiene la función del módulo compilado

    seq1_gpu = cuda.mem_alloc(seq1.nbytes)  # Reserva memoria en la GPU para seq1
    seq2_gpu = cuda.mem_alloc(seq2.nbytes)  # Reserva memoria en la GPU para seq2
    dotplot_gpu = cuda.mem_alloc(dotplot_matrix.nbytes)  # Reserva memoria en la GPU para dotplot_matrix

    cuda.memcpy_htod(seq1_gpu, seq1)  # Copia seq1 a la memoria de la GPU
    cuda.memcpy_htod(seq2_gpu, seq2)  # Copia seq2 a la memoria de la GPU

    block_size = (16, 16, 1)  # Define el tamaño del bloque
    grid_size = (int(np.ceil(len1 / block_size[0])), int(np.ceil(len2 / block_size[1])), 1)  # Define el tamaño de la cuadrícula

    compare_sequences(seq1_gpu, seq2_gpu, dotplot_gpu, np.int32(len1), np.int32(len2), block=block_size, grid=grid_size)  # Llama a la función CUDA

    cuda.memcpy_dtoh(dotplot_matrix, dotplot_gpu)  # Copia el resultado de vuelta a la CPU

    return dotplot_matrix  # Retorna la matriz dotplot

seq1 = np.array(list(merged_sequence_1), dtype=np.chararray)  # Convierte la primera secuencia a un array numpy
seq2 = np.array(list(merged_sequence_2), dtype=np.chararray)  # Convierte la segunda secuencia a un array numpy

start_time = time.time()  # Inicia el temporizador
dotplot = create_dotplot(seq1, seq2)  # Crea el dotplot
end_time = time.time()  # Detiene el temporizador

print(f"Tiempo de ejecución: {end_time - start_time} segundos")  # Imprime el tiempo de ejecución


In [ ]:
# Inicia el tiempo de medición para la operación CUDA
start_time_cuda = time.time()

# Crear el dotplot
create_dotplot(merged_sequence_1, merged_sequence_2)

# Detiene el tiempo de medición para la operación CUDA
end_time_cuda = time.time()

# Calcula el tiempo total de la operación CUDA
time_cuda = end_time_cuda - start_time_cuda

print("Tiempo de ejecución CUDA: ", time_cuda)

Dotplot saved as /content/drive/MyDrive/SEMESTRE 2024-1/PROGRAMACION CONCURRENTE Y DISTRIBUIDA/Proyecto/Proyecto/dotplot_PyCUDA.png
Tiempo de ejecución CUDA:  2.094665050506592


In [ ]:
import cv2
import numpy as np
import math
from threading import Thread

def detect_diagonal_lines(src, dst):
    height, width = src.shape[:2]

    threads = []
    rows_per_thread = height // 4  # Dividir la imagen en 4 secciones para 4 hilos

    for i in range(4):
        start_row = i * rows_per_thread
        end_row = start_row + rows_per_thread if i < 3 else height

        thread = Thread(target=process_rows, args=(src, dst, start_row, end_row))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

def process_rows(src, dst, start_row, end_row):
    for y in range(start_row, end_row):
        for x in range(src.shape[1]):
            gx = int(src[y, x + 1]) - int(src[y, x - 1]) if 0 < x < src.shape[1] - 1 else 0
            gy = int(src[y + 1, x]) - int(src[y - 1, x]) if 0 < y < src.shape[0] - 1 else 0

            magnitude = math.sqrt(gx ** 2 + gy ** 2)
            angle = math.atan2(gy, gx)

            if abs(angle - math.pi / 4) < math.pi / 8 or abs(angle + math.pi / 4) < math.pi / 8:
                dst[y, x] = 0  # Píxel negro para líneas diagonales
            else:
                dst[y, x] = 255  # Píxel blanco para áreas no diagonales

if __name__ == "__main__":
    src = cv2.imread("dotplot_pycuda.png", cv2.IMREAD_GRAYSCALE)
    dst = src.copy()

    detect_diagonal_lines(src, dst)

    cv2.imwrite("resultado.jpg", dst)

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
import numpy as np  # Importa la librería numpy para manipulación de arrays
import time  # Importa el módulo time para medir tiempos de ejecución
import matplotlib.pyplot as plt  # Importa matplotlib para generar gráficos
import json  # Importa json para manejar archivos JSON
from Bio import SeqIO  # Importa SeqIO de Biopython para manipular archivos de secuencias
import pycuda.autoinit  # Inicializa PyCUDA automáticamente
import pycuda.driver as cuda  # Importa el controlador CUDA de PyCUDA
from pycuda.compiler import SourceModule  # Importa SourceModule para compilar y cargar módulos CUDA


# Función para cargar secuencias de archivos FASTA
def load_sequence(filepath):
    """Carga una secuencia de un archivo FASTA."""
    with open(filepath, "r") as file:  # Abre el archivo FASTA en modo lectura
        for record in SeqIO.parse(file, "fasta"):  # Parsea el archivo FASTA y itera sobre cada registro
            return str(record.seq)  # Retorna la secuencia como cadena
    return ""  # Retorna una cadena vacía si no se encuentra ninguna secuencia

# Función para ejecutar el algoritmo en PyCUDA
def execute_py_cuda_algorithm(seq1, seq2):
    """Ejecuta el algoritmo de dotplot en PyCUDA."""
    n = len(seq1)  # Longitud de la primera secuencia
    m = len(seq2)  # Longitud de la segunda secuencia

    dotplot = np.zeros((n, m), dtype=np.uint8)  # Inicializa la matriz dotplot con ceros

    # Convertir las secuencias a arrays de bytes
    seq1_bytes = np.array(list(seq1.encode('ascii')), dtype=np.uint8)
    seq2_bytes = np.array(list(seq2.encode('ascii')), dtype=np.uint8)

    # Reservar memoria en la GPU
    seq1_gpu = cuda.mem_alloc(seq1_bytes.nbytes)
    seq2_gpu = cuda.mem_alloc(seq2_bytes.nbytes)
    dotplot_gpu = cuda.mem_alloc(dotplot.nbytes)

    # Copiar datos a la GPU
    cuda.memcpy_htod(seq1_gpu, seq1_bytes)
    cuda.memcpy_htod(seq2_gpu, seq2_bytes)
    cuda.memcpy_htod(dotplot_gpu, dotplot)

    # Código CUDA para generar el dotplot
    kernel_code = """
    __global__ void generate_dotplot(unsigned char *seq1, unsigned char *seq2, unsigned char *dotplot, int n, int m) {
        int idx = threadIdx.x + blockDim.x * blockIdx.x;
        int idy = threadIdx.y + blockDim.y * blockIdx.y;
        if (idx < n && idy < m) {
            dotplot[idx * m + idy] = (seq1[idx] == seq2[idy]);
        }
    }
    """

    mod = SourceModule(kernel_code)  # Compila el código CUDA
    generate_dotplot = mod.get_function("generate_dotplot")  # Obtiene la función CUDA

    # Ejecutar la función CUDA
    block_size = (16, 16, 1)  # Define el tamaño del bloque
    grid_size = (int(np.ceil(n / 16)), int(np.ceil(m / 16)), 1)  # Define el tamaño de la cuadrícula
    generate_dotplot(seq1_gpu, seq2_gpu, dotplot_gpu, np.int32(n), np.int32(m), block=block_size, grid=grid_size)

    # Copiar resultados de vuelta a la CPU
    cuda.memcpy_dtoh(dotplot, dotplot_gpu)

    return dotplot

# Función para visualizar y guardar el dotplot
def visualize_dotplot(dotplot, estrategia):
    """Visualiza y guarda el dotplot generado."""
    output_file = f"dotplot_{estrategia}.png"  # Nombre del archivo de salida
    plt.imshow(dotplot, cmap='gray_r', interpolation='none')  # Muestra el dotplot
    plt.title(f"Dotplot {estrategia}")  # Título del gráfico
    plt.xlabel('Sequence 2')  # Etiqueta del eje X
    plt.ylabel('Sequence 1')  # Etiqueta del eje Y
    plt.savefig(output_file)  # Guarda el gráfico como archivo PNG
    plt.close()  # Cierra la figura
    print(f"Dotplot saved as {output_file}")  # Imprime un mensaje indicando que el archivo se ha guardado
    return output_file  # Retorna el nombre del archivo de salida

# Celda 3: Función para medir tiempos
def measure_times():
    tiempos = {}

    # Carga de datos
    start_time = time.time()
    # Cargar las secuencias
    Secuencia1 = load_sequence(file1)
    Secuencia2 = load_sequence(file2)
    tiempos['carga_datos'] = time.time() - start_time

    # Usar solo las primeras bases según el filtro
    Secuencia1 = Secuencia1[:filter_size]
    Secuencia2 = Secuencia2[:filter_size]

    # Ejecución del algoritmo en PyCUDA
    start_time = time.time()
    dotplot = execute_py_cuda_algorithm(Secuencia1, Secuencia2)
    tiempos['ejecucion'] = time.time() - start_time

    # Generación de la imagen
    start_time = time.time()
    visualize_dotplot(dotplot, 'pycuda')
    tiempos['visualizacion'] = time.time() - start_time

    # Calcular tiempos totales y tiempos muertos
    tiempos['total'] = tiempos['carga_datos'] + tiempos['ejecucion'] + tiempos['visualizacion']
    tiempos['tiempo_muerto'] = tiempos['total'] - tiempos['ejecucion']

    return tiempos

# Ejecución y medición
file1 = "E_coli.fna"
file2 = "Salmonella.fna"
filter_size = 40000

tiempos = measure_times()
print(tiempos)

# Celda 4: Cálculo de Aceleración y Eficiencia
# Tiempos secuenciales proporcionados
tiempos_secuenciales = {
    "carga_datos": 0.1113274097442627,
    "ejecucion": 45.34779953956604,
    "visualizacion": 1.8736939430236816,
    "total": 47.332820892333984
}

def calculate_speedup_and_efficiency(secuencial_times, paralel_times):
    speedup = secuencial_times['total'] / paralel_times['total']
    efficiency = speedup / num_cores  # num_cores debe estar definido con el número de núcleos usados
    return speedup, efficiency

# Calcular aceleración y eficiencia
num_cores = 1  # Cambiar según la configuración de PyCUDA utilizada
speedup, efficiency = calculate_speedup_and_efficiency(tiempos_secuenciales, tiempos)
print(f"Aceleración: {speedup}")
print(f"Eficiencia: {efficiency}")

# Celda 5: Escalabilidad
def measure_scalability(core_configs):
    scalability_results = []
    for cores in core_configs:
        num_cores = cores
        tiempos = measure_times()
        speedup, efficiency = calculate_speedup_and_efficiency(tiempos_secuenciales, tiempos)
        scalability_results.append({
            "cores": cores,
            "tiempos": tiempos,
            "speedup": speedup,
            "efficiency": efficiency
        })
    return scalability_results

core_configs = [1, 2, 4 ]  # Ejemplos de configuraciones de núcleos
scalability_results = measure_scalability(core_configs)

# Guardar resultados
with open('scalability_results.json', 'w') as f:
    json.dump(scalability_results, f, indent=4)

# Visualizar resultados
for result in scalability_results:
    print(f"Cores: {result['cores']}, Speedup: {result['speedup']}, Efficiency: {result['efficiency']}")

Dotplot saved as dotplot_pycuda.png
{'carga_datos': 0.07873749732971191, 'ejecucion': 3.5503201484680176, 'visualizacion': 16.252521276474, 'total': 19.88157892227173, 'tiempo_muerto': 16.33125877380371}
Aceleración: 2.380737519760608
Eficiencia: 2.380737519760608
Dotplot saved as dotplot_pycuda.png


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def filter_diagonal(dotplot):
    """Filtra y resalta la diagonal del dotplot."""
    n, m = dotplot.shape
    diagonal = np.zeros_like(dotplot)

    for i in range(min(n, m)):
        if dotplot[i, i] == 1:
            diagonal[i, i] = 1

    return diagonal

def visualize_dotplot_with_diagonal(dotplot, diagonal):
    """Visualiza el dotplot con la diagonal resaltada."""
    plt.figure(figsize=(10, 10))
    plt.imshow(dotplot, cmap='gray_r', interpolation='none')
    plt.imshow(diagonal, cmap='Reds', interpolation='none', alpha=0.5)
    plt.title("Dotplot con Diagonal Resaltada")
    plt.xlabel('Sequence 2')
    plt.ylabel('Sequence 1')
    plt.show()

# Generar y visualizar el dotplot con la diagonal resaltada
diagonal = filter_diagonal(dotplot)
visualize_dotplot_with_diagonal(dotplot, diagonal)


NameError: name 'dotplot' is not defined